Figure 3,4

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
from nbodykit.lab import *
from nbodykit import setup_logging, style
from nbodykit.cosmology import Planck15
from scipy.interpolate import InterpolatedUnivariateSpline as ius
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # many annoying dask warnings from bigfile
plt.style.use(style.notebook)
setup_logging()

### Quijote (Rockstar for conc.)

In [ ]:
quijote_path = "/pscratch/sd/j/jsull/Quijote_png/quijote_s8s/"
s8strings = ["s8_m/", "s8_0/","s8_p/"] # s8_0 is not the first element!
s8vals = np.array([0.819,0.834, 0.849]) #from table here: https://quijote-simulations.readthedocs.io/en/latest/types.html
seeds = np.arange(100,131)
# they have output redshifts at 
zidxs = [0,1,2,3,4]
avals = np.array([0.25, 0.333333, 0.5,0.666667,1.0])
zvals = 1./avals - 1.0
L = 1000.0

In [ ]:
# Load rockstar catalogs (from https://quijote-simulations.readthedocs.io/en/latest/halos.html?highlight=rockstar)
def get_rockstar_quijote(s8idx,seed,zidx):
    # catalog file
    f_catalog = quijote_path+s8strings[s8idx]+'{0}/out_{1}_pid.list'.format(seed,zidx)

    # get the column names
    with open(f_catalog) as f: colnames = f.readline()
    colkeys = colnames[1:-1].split(' ')
    # read the halo catalog
    data = np.loadtxt(f_catalog)

    return dict(zip(colkeys, data.T))

In [4]:
def get_rockstar_quijote_allseeds(s8idx,zidx):
    return [get_rockstar_quijote(s8idx,seed,zidx) for seed in seeds]

In [ ]:
# bin by mass (on a single catalog)
def bin_prop(cat,bin_edges,props=['Mvir']):
    # Returns a 1D/2D array of binned catalogs by up to two properties
    prop_cats = []
    if len(props==1):
        for prop in props:
            col = cat[prop]
            for be in bin_edges[prop]:
                prop_mask = (col >= be[0]) & (col < be[1])
                prop_cats.append(cat[prop][prop_mask])
    elif len(props==2):
        # cut by two properties at once
        prop_masks = [[] for _ in range(len(props))]
        for prop in props:
            col = cat[prop]
            for be in bin_edges[prop]:
                prop_masks[prop].append((col >= be[0]) & (col < be[1]))
        for i in range(len(prop_masks[0])):
            for j in range(len(prop_masks[1])):
                prop_cats.append(cat[prop_masks[0][i] & prop_masks[1][j]])
        for prop in props:
            col = cat[prop]
            for be in bin_edges[prop]:
                prop_mask = (col >= be[0]) & (col < be[1])
                prop_cats.append(cat[prop][prop_mask])
    else: return NotImplementedError("FIXME ADD MORE PARAMETERS")
    return prop_cats

def bin_mass_allzs(zcats,bin_edges,prop):
    return [bin_prop(cat,bin_edges,prop) for cat in zcats]

def bin_mass_allseeds_allzs(seedcats,bin_edges,prop):
    return [bin_mass_allzs(scat,bin_edges,prop) for scat in seedcats]

# compute the number densities
def compute_nbar_seedmean_z(cats,L=L):
    # cats a nested list with outer dim zs, inner dim seeds
    V = L**3
    mean_nbars,var_nbars = [],[]
    for zidx in zidxs:
            mean_nbars.append(np.mean([len(cats[zidx][sidx]['ID'])/V for sidx in range(len(seeds))]))
            var_nbars.append(np.var([len(cats[zidx][sidx]['ID'])/V for sidx in range(len(seeds))]))
            print("z={0:.2f}, nbar={1:.2e}".format(zvals[zidx],mean_nbars[zidx]))
    return mean_nbars,var_nbars

def compute_nbars_z_mass(cats,mass,L=L):
    # cats a nested list with outer dim zs, inner dim seeds
    V = L**3
    nbars = np.zeros((len(zidxs),len(seeds)))
    for zidx in zidxs:
        for sidx in range(len(seeds)):
            mask = (cats[zidx][sidx]['Mvir'] > mass[0]) & (cats[zidx][sidx]['Mvir'] < mass[1])
            nbars[zidx,sidx] = len(cats[zidx][sidx]['ID'][mask])/V
    return nbars

def compute_nbars_z_mass_conc(cats,mass,conc,L=L):
    # cats a nested list with outer dim zs, inner dim seeds
    V = L**3
    nbars = np.zeros((len(zidxs),len(seeds)))
    for zidx in zidxs:
        for sidx in range(len(seeds)):
            cats[zidx][sidx]['conc'] = cats[zidx][sidx]['Rvir']/cats[zidx][sidx]['Rs']
            mass_mask = (cats[zidx][sidx]['Mvir'] > mass[0]) & (cats[zidx][sidx]['Mvir'] < mass[1])
            conc_mask = (cats[zidx][sidx]['conc'] > conc[0]) & (cats[zidx][sidx]['conc'] < conc[1])
            mask = mass_mask & conc_mask
            nbars[zidx,sidx] = len(cats[zidx][sidx]['ID'][mask])/V
    return nbars

In [ ]:
#takes 12 min or so...
rockstar_s8m_allseeds_allz = [get_rockstar_quijote_allseeds(0,zidx) for zidx in zidxs]
rockstar_s80_allseeds_allz = [get_rockstar_quijote_allseeds(1,zidx) for zidx in zidxs]
rockstar_s8p_allseeds_allz = [get_rockstar_quijote_allseeds(2,zidx) for zidx in zidxs]

In [7]:
rockstar_s80_100_allz = [get_rockstar_quijote(0,100,zidx) for zidx in zidxs]

In [ ]:
# selection on mass
halo_mass_bins = np.logspace(13.3,15.0,10+1)
halo_mass_cens = (halo_mass_bins[1:]+halo_mass_bins[:-1])/2
halo_mass_bin_edges = list(zip(halo_mass_bins[:-1],halo_mass_bins[1:]))

In [22]:
nbarmass_halos_s80 = np.array([compute_nbars_z_mass(rockstar_s80_allseeds_allz, halo_mass_bin_edges[midx]) for midx in range(len(halo_mass_bin_edges))])
nbarmass_halos_s8m = np.array([compute_nbars_z_mass(rockstar_s8m_allseeds_allz, halo_mass_bin_edges[midx]) for midx in range(len(halo_mass_bin_edges))])
nbarmass_halos_s8p = np.array([compute_nbars_z_mass(rockstar_s8p_allseeds_allz, halo_mass_bin_edges[midx]) for midx in range(len(halo_mass_bin_edges))])


In [ ]:
mean_bphiz_mass_halos_zspline_s8m = np.array([np.mean([ius(zvals[::-1],nbarmass_halos_s8m[midx][::-1,sidx]).derivative()(zvals)*-2*(1+zvals)/nbarmass_halos_s8m[midx][:,sidx] for sidx in range(len(seeds))],axis=0) for midx in range(len(halo_mass_bin_edges))])
mean_bphiz_mass_halos_zspline_s80 = np.array([np.mean([ius(zvals[::-1],nbarmass_halos_s80[midx][::-1,sidx]).derivative()(zvals)*-2*(1+zvals)/nbarmass_halos_s80[midx][:,sidx] for sidx in range(len(seeds))],axis=0) for midx in range(len(halo_mass_bin_edges))])
mean_bphiz_mass_halos_zspline_s8p = np.array([np.mean([ius(zvals[::-1],nbarmass_halos_s8p[midx][::-1,sidx]).derivative()(zvals)*-2*(1+zvals)/nbarmass_halos_s8p[midx][:,sidx] for sidx in range(len(seeds))],axis=0) for midx in range(len(halo_mass_bin_edges))])
var_bphiz_mass_halos_zspline_s8m = np.array([np.var([ius(zvals[::-1],nbarmass_halos_s8m[midx][::-1,sidx]).derivative()(zvals)*-2*(1+zvals)/nbarmass_halos_s8m[midx][:,sidx] for sidx in range(len(seeds))],axis=0) for midx in range(len(halo_mass_bin_edges))])
var_bphiz_mass_halos_zspline_s80 = np.array([np.var([ius(zvals[::-1],nbarmass_halos_s80[midx][::-1,sidx]).derivative()(zvals)*-2*(1+zvals)/nbarmass_halos_s80[midx][:,sidx] for sidx in range(len(seeds))],axis=0) for midx in range(len(halo_mass_bin_edges))])
var_bphiz_mass_halos_zspline_s8p = np.array([np.var([ius(zvals[::-1],nbarmass_halos_s8p[midx][::-1,sidx]).derivative()(zvals)*-2*(1+zvals)/nbarmass_halos_s8p[midx][:,sidx] for sidx in range(len(seeds))],axis=0) for midx in range(len(halo_mass_bin_edges))])

In [ ]:
nbarallhalos_s8m,varallhalos_s8m = compute_nbar_seedmean_z(rockstar_s8m_allseeds_allz)
nbarallhalos_s80,varallhalos_s80 = compute_nbar_seedmean_z(rockstar_s80_allseeds_allz)
nbarallhalos_s8p,varallhalos_s8p = compute_nbar_seedmean_z(rockstar_s8p_allseeds_allz)

## Mass bins

In [26]:
# and concentration
conc_bins = np.linspace(1,20,10+1)
conc_cens = (conc_bins[1:]+conc_bins[:-1])/2
conc_bin_edges = list(zip(conc_bins[:-1],conc_bins[1:]))

In [27]:
nbarmassconc_halos_s8m = np.array([[compute_nbars_z_mass_conc(rockstar_s8m_allseeds_allz, halo_mass_bin_edges[midx],conc_bin_edges[cidx]) for midx in range(len(halo_mass_bin_edges))] for cidx in range(len(conc_bin_edges))])
nbarmassconc_halos_s80 = np.array([[compute_nbars_z_mass_conc(rockstar_s80_allseeds_allz, halo_mass_bin_edges[midx],conc_bin_edges[cidx]) for midx in range(len(halo_mass_bin_edges))] for cidx in range(len(conc_bin_edges))])
nbarmassconc_halos_s8p = np.array([[compute_nbars_z_mass_conc(rockstar_s8p_allseeds_allz, halo_mass_bin_edges[midx],conc_bin_edges[cidx]) for midx in range(len(halo_mass_bin_edges))] for cidx in range(len(conc_bin_edges))])


In [ ]:
# copy formula from other notebook...
def bphi(Ns,Xs): # from TNG file
    #X = sigma8 or Dz
    # Function to compute (centered) FD derivative given some numberdensity tuple (Nm,N0,Np)
    # and corresponding redshift/s8 tuple (zm,z0,zp) in order of decreasing redshift (increasing a)
    # or increasing sigma8
    d1pX1,d1pX2 = Xs[0]-Xs[1],-(Xs[1]-Xs[2])
    reld1pX1,reld1pX2 = d1pX1/(Xs[1]),d1pX2/(Xs[1])
    bphi_hi = 2/reld1pX1 *(Ns[0]/Ns[1] -1) #low on N refers to low g-r, redshifts in dec order
    bphi_lo = 2/reld1pX2 *(Ns[2]/Ns[1] -1) 
    #averaging procedure of Barriera++20
    bphi = (bphi_hi + bphi_lo)/2.0
    return bphi



In [ ]:
mean_s8_su_bphi_mass = np.array([ [np.mean([bphi([nbarmass_halos_s8m[midx,zidx,sidx],nbarmass_halos_s80[midx,zidx,sidx],nbarmass_halos_s8p[midx,zidx,sidx]],s8vals) for sidx in range(len(seeds)) ] ) for zidx in zidxs] for midx in range(len(halo_mass_bin_edges))])
var_s8_su_bphi_mass = np.array([ [np.var([bphi([nbarmass_halos_s8m[midx,zidx,sidx],nbarmass_halos_s80[midx,zidx,sidx],nbarmass_halos_s8p[midx,zidx,sidx]],s8vals) for sidx in range(len(seeds)) ] ) for zidx in zidxs] for midx in range(len(halo_mass_bin_edges))])


In [ ]:
mean_s8_su_bphi_massconc = np.array([[ [np.mean([bphi([nbarmassconc_halos_s8m[cidx,midx,zidx,sidx],nbarmassconc_halos_s80[cidx,midx,zidx,sidx],nbarmassconc_halos_s8p[cidx,midx,zidx,sidx]],s8vals) for sidx in range(len(seeds)) ] ) for zidx in zidxs] for midx in range(len(halo_mass_bin_edges))] for cidx in range(len(conc_bin_edges))])
var_s8_su_bphi_massconc = np.array([[ [np.var([bphi([nbarmassconc_halos_s8m[cidx,midx,zidx,sidx],nbarmassconc_halos_s80[cidx,midx,zidx,sidx],nbarmassconc_halos_s8p[cidx,midx,zidx,sidx]],s8vals) for sidx in range(len(seeds)) ] ) for zidx in zidxs] for midx in range(len(halo_mass_bin_edges))] for cidx in range(len(conc_bin_edges))])


In [ ]:
mean_bphiz_massconc_halos_zspline_s80 = np.array([[np.mean([ius(zvals[::-1],nbarmassconc_halos_s80[cidx,midx][::-1,sidx]).derivative()(zvals)*-2*(1+zvals)/nbarmassconc_halos_s80[cidx,midx][:,sidx] for sidx in range(len(seeds))],axis=0) for midx in range(len(halo_mass_bin_edges))]for cidx in range(len(halo_mass_bin_edges))])
var_bphiz_massconc_halos_zspline_s80 = np.array([[np.var([ius(zvals[::-1],nbarmassconc_halos_s80[cidx,midx][::-1,sidx]).derivative()(zvals)*-2*(1+zvals)/nbarmassconc_halos_s80[cidx,midx][:,sidx] for sidx in range(len(seeds))],axis=0) for midx in range(len(halo_mass_bin_edges))]for cidx in range(len(halo_mass_bin_edges))])


In [33]:
fzvals = Planck15.scale_independent_growth_rate(zvals)

In [34]:
Plin = cosmology.LinearPower(Planck15, redshift=0.0, transfer='CLASS')

In [ ]:
for u_zidx in range(len(zvals)):
    plt.figure(figsize=(5,4))
    plt.title("z = {0:.1f}".format(zvals[u_zidx]))
    plt.errorbar(halo_mass_cens,mean_s8_su_bphi_mass[:,u_zidx],yerr=np.sqrt(var_s8_su_bphi_mass[:,u_zidx]),fmt='s',capsize=5,capthick=2,label=r'$b_{\phi}^{\sigma_8}$')
    plt.errorbar(halo_mass_cens,mean_bphiz_mass_halos_zspline_s80[:,u_zidx],yerr=np.sqrt(var_bphiz_mass_halos_zspline_s80[:,u_zidx]),fmt='o',capsize=5,capthick=2,label=r'$b_{\phi}^{z}$')
    plt.errorbar(halo_mass_cens,mean_bphiz_mass_halos_zspline_s80[:,u_zidx]/fzvals[u_zidx],yerr=np.sqrt(var_bphiz_mass_halos_zspline_s80[:,u_zidx])/fzvals[u_zidx],fmt='.',capsize=5,capthick=2,label=r'$b_{\phi}^{D(z)} = f^{-1}~b_{\phi}^{z}$')
    plt.legend(prop={"size":14})
    plt.xscale('log')
    plt.xlabel(r'$M_{h}$')
    plt.ylabel(r'$b_{\phi}^{X}$')
    plt.savefig("../plots/bphi_mass_z_{0:.1f}_quijote.pdf".format(zvals[u_zidx]))
    plt.savefig("../plots/bphi_mass_z_{0:.1f}_quijote.png".format(zvals[u_zidx]))
    plt.show()

# Try a selection on just concentration

In [18]:
import matplotlib.pylab as pl
colors = pl.cm.Blues(np.linspace(0.2,1,len(conc_cens)))

In [ ]:
for u_zidx in range(2,len(zvals)): 
    for m_idx in range(len(halo_mass_cens)):
        plt.figure(figsize=(5,4))
        plt.title(r"$\langle M_{h} \rangle$"+" = {0:.1e}, ".format(halo_mass_cens[m_idx])+r"z = "+" {0:.1f}".format(zvals[u_zidx]),fontsize=18)
        plt.errorbar(conc_cens,mean_s8_su_bphi_massconc[:,m_idx,u_zidx],yerr=np.sqrt(var_s8_su_bphi_massconc[:,m_idx,u_zidx]),fmt='s',capsize=5,capthick=2,
                     label=r'$b_{\phi}^{\sigma_8}$' 
                     )
        plt.errorbar(conc_cens,mean_bphiz_massconc_halos_zspline_s80[:,m_idx,u_zidx]/fzvals[u_zidx],yerr=np.sqrt(var_bphiz_massconc_halos_zspline_s80[:,m_idx,u_zidx])/fzvals[u_zidx],
                    fmt='o',capsize=5,capthick=2,label=None,

                    )
        plt.errorbar(conc_cens,mean_bphiz_massconc_halos_zspline_s80[:,m_idx,u_zidx]/fzvals[u_zidx],yerr=np.sqrt(var_bphiz_massconc_halos_zspline_s80[:,m_idx,u_zidx])/fzvals[u_zidx],
                    fmt='o',capsize=5,capthick=2,label=r'$b_{\phi}^{D(z)} = f^{-1}~b_{\phi}^{z}$',
                    
                    )
        plt.legend()
        plt.xlabel(r'$c$')
        plt.ylabel(r'$b_{\phi}^{X}$')
        plt.savefig('../plots/bphiczm_z_{0:.1f}_m_{1:.3e}conc_quijote.pdf'.format(zvals[u_zidx],halo_mass_cens[m_idx]))
        plt.savefig('../plots/bphiczcm_z_{0:.1f}_m_{1:.3e}conc_quijote.png'.format(zvals[u_zidx],halo_mass_cens[m_idx]))
        plt.show()